# Process VIIRS Night-time lights data

The data we use are the monthly DNB (day-night band) composites.

Download these from 
https://www.ngdc.noaa.gov/eog/viirs/download_dnb_composites.html

There are 6 tiles for each global monthly image; they are at 500m resolution. They are provided in .tgz files and they are very large due to the resolution and the nature of the data meaning they do not compress well. 

This notebook helps organise the downloaded files, unzip, and merge them.

In [1]:
import os

In [2]:
from osgeo import gdal

In [3]:
import zipfile
import tarfile

In [4]:
import subprocess

In [5]:
import glob
import shutil

In [11]:
from joblib import Parallel, delayed


In [6]:
indir = r'E:\Temp\nightlights'


Move all the files from a single download folder into subfolders corresponding to the date

In [7]:
infiles = glob.glob(indir+'\\*.tgz')

In [9]:
for f in infiles:
    base = os.path.basename(f)
    daterange = base.split('_')[2]
    destdir = os.path.join(indir,daterange)
    if not os.path.exists(destdir):
        os.mkdir(destdir)
    shutil.move(f, destdir)
    

In [10]:
indirs = [d for d in glob.glob(indir+"\\*") if os.path.isdir(d)]


In [12]:
sorted(indirs)

['E:\\Temp\\nightlights\\20160501-20160531',
 'E:\\Temp\\nightlights\\20160601-20160630',
 'E:\\Temp\\nightlights\\20160701-20160731',
 'E:\\Temp\\nightlights\\20160801-20160831',
 'E:\\Temp\\nightlights\\20160901-20160930']

Define a function which mosaics, translates, and prepares tiles into a single output tiff. We just do this by shelling out to the gdal command line tools.

In [16]:
vrtcommand = "gdalbuildvrt {0} {1}"
transcommand = "gdal_translate -of GTiff -co COMPRESS=LZW "+\
    "-co PREDICTOR=2 -co TILED=YES -co SPARSE_OK=TRUE -co BIGTIFF=YES "+\
    "--config GDAL_CACHEMAX 8000 {0} {1}"
ovcommand = "gdaladdo -ro --config COMPRESS_OVERVIEW LZW --config USE_RRD NO " +\
        "--config TILED YES {0} 2 4 8 16 32 64 128 256 --config GDAL_CACHEMAX 8000"
statcommand = "gdalinfo -stats {0} >nul"
def translateDir(dIn, dOut=None):
    if dOut is None:
        dOut = dIn
    print "Operating in folder "+dIn
    filetag = 'vcmslcfg' # or vcmcfg for stray-light-excluded
    tars = glob.glob(dIn+"\\"+"*"+filetag+"*.tgz")
    if len(tars)<6:
        print "incomplete folder!"
        return
    dirTileList = []
    subDirName = os.path.basename(dIn)
    outFile = os.path.join(dOut, "VIIRS.DNB.StrayLightCorrected."+subDirName)
    
    if os.path.exists(outFile + ".tif"):
        print "Already done! Skipping"
        return
    #print outFile + " doesn't exist: running!"
    #return
    for t in tars:
        if t.find(filetag) == -1:
            continue
        tar = tarfile.open(t)
        #tardir = os.path.dirname(t)
        members = tar.members
        for m in members:
            if m.name.find('avg_rad') != 1:
                if os.path.exists(os.path.join(dIn, m.name)):
                    print m.name +" exists - skipping extraction!"
                    continue
                print "Extracting "+m.name
                
                tar.extract(m, dIn)
                dirTileList.append(os.path.join(dIn, m.name))
    
    
    vrtBuilder = vrtcommand.format(outFile+".vrt", os.path.join(dIn,"*"+filetag+"*.tif"))
    print vrtBuilder
    subprocess.call(vrtBuilder)
        
    transBuilder = transcommand.format(outFile+".vrt", outFile+".tif")
    print transBuilder
    subprocess.call(transBuilder)
    
    ovBuilder = ovcommand.format(outFile+".tif")
    print ovBuilder
    subprocess.call(ovBuilder)
    
    statBuilder = statcommand.format(outFile+".tif")
    print statBuilder
    subprocess.call(statBuilder)
    
    for f in dirTileList:
        os.remove(f)

now just run it on each subfolder, or some of them, or whatever. Tried using the Parallel library to run multiple at once but it didn't work, or something, I can't remember

In [ ]:
#Parallel(n_jobs=4)(delayed(translateDir)(d) for d in indirs)
outDir = r'\\map-fs1.ndph.ox.ac.uk\map_data\mastergrids\Other_Global_Covariates\NightTimeLights\VIIRS_DNB_Monthly'
outDir = r'C:\Temp\nightlights'
for d in sorted(indirs[1:]):
    translateDir(d, outDir)